**MARKET BASKET ANALYSIS**

*By Sadhanha Anand*


In [ ]:
# Install the necessary library
!pip install apyori

# Import necessary libraries
import pandas as pd
from apyori import apriori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=98b8975d35583341ded2b40428fda85b02ac8da0b3c3e820802daeda9d44a919
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [ ]:
# Function to safely load CSV files
def load_csv(filename, expected_columns):
    try:
        data = pd.read_csv(filename, on_bad_lines='skip')
        print(f"Successfully loaded {filename}.")
        if data.shape[1] != expected_columns:
            print(f"Warning: Some columns may be missing or extra in {filename}. Expected {expected_columns}, found {data.shape[1]} columns.")
    except Exception as e:
        print(f"Failed to load {filename} due to an error: {e}")
        return None
    return data

In [ ]:
# Load the datasets with expected column counts for verification
products = load_csv('products.csv', 4)  # product_id, product_name, aisle_id, department_id
order_products_train = load_csv('order_products__train.csv', 4)  # order_id, product_id, add_to_cart_order, reordered
orders = load_csv('orders.csv', 7)  # order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order

Successfully loaded products.csv.
Successfully loaded order_products__train.csv.
Successfully loaded orders.csv.


In [ ]:
# Check if any file failed to load
if products is None or order_products_train is None or orders is None:
    raise SystemExit("One or more files failed to load, please check the files and try again.")

# Merge order_products_train with products to link product names with each order
merged_data = pd.merge(order_products_train[['order_id', 'product_id']], products[['product_id', 'product_name']], on='product_id')

In [ ]:
# Group the data by order_id and collect all product names associated with each order
transactions = merged_data.groupby('order_id')['product_name'].apply(list).tolist()

# Specify the minimum support and confidence values
min_support_value = 0.01
min_confidence_value = 0.1

# Run the Apriori algorithm to find frequent itemsets and association rules
rules = list(apriori(transactions, min_support=min_support_value, min_confidence=min_confidence_value, min_lift=1.0))

# Print out the discovered rules
print("Association Rules:")
for rule in rules:
    base_items = ', '.join(rule.items)
    support = rule.support
    for stats in rule.ordered_statistics:
        before = ', '.join(stats.items_base)
        after = ', '.join(stats.items_add)
        confidence = stats.confidence
        lift = stats.lift
        print(f"If a customer buys [{before}], they might also buy [{after}]")
        print(f"Support: {support:.4f}, Confidence: {confidence:.4f}")

Association Rules:
If a customer buys [], they might also buy [Bag of Organic Bananas]
Support: 0.1172, Confidence: 0.1172
If a customer buys [], they might also buy [Banana]
Support: 0.1409, Confidence: 0.1409
If a customer buys [Bag of Organic Bananas], they might also buy [Organic Baby Spinach]
Support: 0.0164, Confidence: 0.1402
If a customer buys [Organic Baby Spinach], they might also buy [Bag of Organic Bananas]
Support: 0.0164, Confidence: 0.2213
If a customer buys [Bag of Organic Bananas], they might also buy [Organic Hass Avocado]
Support: 0.0182, Confidence: 0.1556
If a customer buys [Organic Hass Avocado], they might also buy [Bag of Organic Bananas]
Support: 0.0182, Confidence: 0.3267
If a customer buys [Bag of Organic Bananas], they might also buy [Organic Raspberries]
Support: 0.0138, Confidence: 0.1182
If a customer buys [Organic Raspberries], they might also buy [Bag of Organic Bananas]
Support: 0.0138, Confidence: 0.3297
If a customer buys [Bag of Organic Bananas], th

**Recommendations:**

1. **Seasonal Promotions and Themed Campaigns:**
Create seasonal promotions and themed marketing campaigns based on frequently co-purchased items that align with specific seasons or events. For instance, if data shows that "Pumpkin Spice Latte" and "Cinnamon Rolls" are popular in the fall, launch a fall-themed campaign promoting these items together. Similarly, for summer, bundle "Organic Lemonade" and "Grilled Chicken Breasts" for a barbecue promotion. These themed campaigns can attract more customers by tapping into the seasonal appeal and increasing the visibility of related products.


2. **Loyalty Program Enhancements:**
Enhance your loyalty program by integrating the insights from association rules. Offer personalized rewards based on frequent purchase patterns. For example, if a customer regularly buys "Organic Baby Spinach" and "Greek Yogurt," provide loyalty points or discounts on these items after a certain number of purchases. This approach not only incentivizes repeat purchases but also strengthens customer loyalty by offering tangible rewards that align with their shopping habits.